# Predict ED_dispo with result from Lbl2TransformerVec and without

For architecture and weight initialization, we will compare:

• <b> BERT (baseline) 

• BioBERT

• Clinical BioBERT
    
• squeezebert-uncased

• BlueBERT

• XLNet (baseline)

• Clinical XLnet

In [1]:
#!pip install tensorflow_hub

In [2]:
#!pip install tensorflow-text --no-dependencies

In [3]:
import pandas as pd
import numpy as np
import gc
import os
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
import re
import tensorflow as tf
from transformers import TFXLNetModel, XLNetTokenizer
from transformers import AutoTokenizer, AutoModel, TFAutoModel, TFBertModel
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import ParameterSampler
import tensorflow_hub as hub
import json
import csv 

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

import mlflow
import mlflow.keras
import tensorflow_text as text
import seaborn as sns
from sentence_transformers import SentenceTransformer

2023-03-12 14:35:43.130646: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 14:35:43.284861: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-12 14:35:43.898111: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib64:/opt/amazon/openmpi/lib64:/usr/local/cuda/efa/lib:/usr/local/cuda

In [4]:
#!pip install mlflow
#!databricks configure --host https://community.cloud.databricks.com/

In [5]:

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('There are %d GPU(s) available.' % torch.cuda.device_count())
print('We will use the GPU:', torch.cuda.get_device_name(0))

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


2023-03-12 14:35:45.507937: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 14:35:45.542761: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 14:35:45.544764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 14:35:45.545613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there

In [6]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)
tf.device('/device:GPU:0')

Num GPUs Available:  1


2023-03-12 14:35:47.578025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 14:35:47.578893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 14:35:47.579655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 14:35:47.580964: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 14:35:47.581805: I tensorflow/compiler/xla/stream_executo

In [8]:
%env  DATABRICKS_USERNAME= 'karengar@stanford.edu' 
%env  DATABRICKS_PASSWORD = 'Projectcs224*'

env: DATABRICKS_USERNAME='karengar@stanford.edu'
env: DATABRICKS_PASSWORD='Projectcs224*'


 - Import data

In [9]:
data_test = pd.read_csv('/root/data/ed_test.csv')
data_train = pd.read_csv('/root/data/ed_train.csv')
data_val = pd.read_csv('/root/data/ed_val.csv')
data_val.head()

,Unnamed: 0,CSN,Rads_order_ID,Study,Impression,ED_dispo
0,80343,131318610796,755426017,XR ELBOW 3 VIEWS LEFT,1. No displaced fracture or traumatic malalig...,Discharge
1,66645,131314132694,741821327,XR CHEST 1 VIEW,1. Apparent retrocardiac opacity favored to b...,Admit to Inpatient
2,76374,131317332223,751367975,XR CHEST 1 VIEW,1. No acute cardiopulmonary disease.,Discharge
3,66912,131314275465,742214156,XR CHEST 1 VIEW,1. Interval improvement in aeration of the ri...,Admit to Inpatient
4,118086,131329544269,789721134,CT HEAD WO IV CONTRAST,1. No acute intracranial abnormality. 2. Sca...,Discharge


In [10]:
#result_l2v = pd.read_csv('/root/Project-CS224N-ED-Disposition/result_l2v_v1.csv') 
#result_l2v.head()

In [11]:
def plot_roc_curve(y_proba, y_test, labels, mlflow, split_type):
    # Plots the Probability Distributions and the ROC Curves One vs Rest
    fig = plt.figure(figsize = (17, 8))
    bins = [i/20 for i in range(20)] + [1]
    roc_auc_ovr = {}
    for i in range(len(labels)):
        # Gets the class
        c = labels[i]

        # Prepares an auxiliar dataframe to help with the plots
        df_aux = pd.DataFrame()
        df_aux['class'] = [1 if y == c else 0 for y in y_test]
        df_aux['prob'] = y_proba[:, i]
        df_aux = df_aux.reset_index(drop = True)

        # Plots the probability distribution for the class and the rest
        fig_upper = plt.subplot(2, 4, i+1)
        x = df_aux[df_aux['class'] ==0]
        plt.hist(x['prob'], density=True, label='Rest')
        y = df_aux[df_aux['class'] ==1]
        plt.hist(y['prob'], density=True, label=f" {c}", bins = bins)
        plt.title(c)
        plt.legend(loc='upper right')
        plt.xlabel(f"P(x = {c})")
        #plt.show()
        #mlflow.log_figure(fig, f"Histogram: {c}.png") 


        # Calculates the ROC Coordinates and plots the ROC Curves
        fpr, tpr, thresholds = roc_curve(df_aux['class'], df_aux['prob'], pos_label=1)
        mauc = auc(fpr, tpr)
        fig_bottom = plt.subplot(2, 4, i+5)
        plt.title( f"ROC: {c}")
        plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % mauc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        #plt.show()
        mlflow.log_metric("auc_" + split_type + "_"+  c, mauc) 

    plt.tight_layout()
    mlflow.log_figure(fig, "Hist_and_ROC_" + c+ "_"+ split_type + ".png") 
    
def evaluate(y_test, y_pred, mlflow,  split_type):
    """
    Evaluation function. For each of the text in evaluation data, it reads the score from
    the predictions made. And based on this, it calculates the values of
    True positive, True negative, False positive, and False negative.

    :param y_test: true labels
    :param y_pred: predicted labels
    :param labels: list of possible labels
    :return: evaluation metrics for classification like, precision, recall, and f_score
    """
     
    b = np.zeros_like(y_pred)
    b[np.arange(len(y_pred)), y_pred.argmax(1)] = 1
    y_proba = y_pred

    y_pred = pd.DataFrame(b, columns = list(y_test.columns))
    y_pred = y_pred.idxmax(axis=1)
    y_test = y_test.idxmax(axis=1)

    labels = list(y_test.unique())
    labels = sorted(labels)

    plot_roc_curve(y_proba, y_test, labels, mlflow, split_type)

    
    confusion = confusion_matrix(y_test, y_pred)

    # importing accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average= None)
    recall = recall_score(y_test, y_pred, average= None)
    f1score = f1_score(y_test, y_pred, average= None)


    report = classification_report(y_test, y_pred, target_names = labels)
    print(report)
    f_name = 'report_ '+ split_type +'.yaml'
    mlflow.log_dict(report, f_name)
    #mlflow.log_metric("auc_" + split_type, mauc) 
    mlflow.log_metric("accuracy_"+ split_type, accuracy)
   

    return accuracy, precision, recall, f1score




def mlflow_log_parameters(parameter):
    # Log parameters
    mlflow.log_param("dropout", parameter['dropout'])
    mlflow.log_param("learning_rate", parameter['learning_rate'])
    mlflow.log_param("epochs", parameter['epochs'])
    mlflow.log_param("batch_size", parameter['batch_size'])



def get_inputs_biobert(sentences):
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb')
    model = AutoModel.from_pretrained('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb')

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embeddings

def create_bert(bert, learning_rate=1e-3, dropout=0.1):
    input_ids = tf.keras.layers.Input(shape=(158,), dtype=tf.int32, name="input_ids")
    input_mask = tf.keras.layers.Input(shape=(158,), dtype=tf.int32, name="attention_mask")

    embeddings = bert(input_ids, attention_mask = input_mask)[0]
    bert.trainable = False
    out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    y = tf.keras.layers.Dense(4, activation='softmax', name='outputs')(out)

    # Compile model
    model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', 
               metrics=[tf.keras.metrics.AUC(), 'accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model


def training_test_mae( mlflow, history):
    # Plot training and test loss at each epoch 
    fig, ax = plt.subplots()
    ax.plot(history.history['accuracy'], label='Training acc')
    #plt.plot(history.history['val_accuracy'], label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.legend()
    plt.show()
    mlflow.log_figure(fig, "training_validation_accuracy.png") 

    fig, ax = plt.subplots()
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    #plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    mlflow.log_figure(fig, "training_validation_loss.png") 


# Scenario 1

- Without using results from l2v

In [12]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/karengar@stanford.edu/BERT_l2v")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/1072956514573773', creation_time=1678354854333, experiment_id='1072956514573773', last_update_time=1678578685444, lifecycle_stage='active', name='/Users/karengar@stanford.edu/BERT_l2v', tags={'mlflow.experiment.sourceName': '/Users/karengar@stanford.edu/BERT_l2v',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'karengar@stanford.edu',
 'mlflow.ownerId': '3913783827154434'}>

In [13]:
#data[list(result_l2v.iloc[:, 3:].columns)] = result_l2v.iloc[:, 3:]
#data.head()

In [14]:
X_train = data_train.drop(['Unnamed: 0', 'ED_dispo', 'CSN', 'Rads_order_ID'], axis=1)
X_test =  data_test.drop(['Unnamed: 0', 'ED_dispo', 'CSN', 'Rads_order_ID'], axis=1)
X_val = data_val.drop(['Unnamed: 0', 'ED_dispo', 'CSN', 'Rads_order_ID'], axis=1)

# Assigning numerical values and storing in another column
y_train =  pd.get_dummies(data_train['ED_dispo'])
y_test = pd.get_dummies(data_test['ED_dispo'])[list(y_train.columns)]
y_val =  pd.get_dummies(data_val['ED_dispo'])[list(y_train.columns)]



In [15]:
del data_train, data_test, data_val 

In [16]:
rng = np.random.RandomState(0)
param_dist = {    'dropout': [0.1,0.2, 0.3],
                  'learning_rate': loguniform.rvs(1e-3, 1e-1, size= 10),
                  'epochs': [1],#[5, 10],
                  'batch_size':[64, 128]
                  }
            
dict_parameters = ParameterSampler(param_distributions=param_dist, n_iter= 10, random_state=rng)


In [17]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFAutoModel.from_pretrained("bert-base-cased")

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


2023-03-12 14:35:48.678570: I tensorflow/core/common_runtime/placer.cc:114] input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.678605: I tensorflow/core/common_runtime/placer.cc:114] _EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.678623: I tensorflow/core/common_runtime/placer.cc:114] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.683844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 177.00M (185597952 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-03-12 14:35:48.688483: I tensorflow/core/common_runtime/placer.cc:114] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.688509: I tensorflow/core/common_runtime/placer.cc:114] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.690270: I tensorflow/core/common_runtime/placer.cc:114] r

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:loc

2023-03-12 14:35:48.971663: I tensorflow/core/common_runtime/placer.cc:114] input: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-03-12 14:35:48.971709: I tensorflow/core/common_runtime/placer.cc:114] _EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.971720: I tensorflow/core/common_runtime/placer.cc:114] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.980487: I tensorflow/core/common_runtime/placer.cc:114] input: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-03-12 14:35:48.980525: I tensorflow/core/common_runtime/placer.cc:114] Shape: (Shape): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.980535: I tensorflow/core/common_runtime/placer.cc:114] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-03-12 14:35:48.984378: I tensorflow/core/common_runtime/placer.cc:114] dims: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-03-12 14

ResourceExhaustedError: Exception encountered when calling layer 'bert' (type TFBertMainLayer).

{{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]

Call arguments received by layer 'bert' (type TFBertMainLayer):
  • input_ids=tf.Tensor(shape=(3, 5), dtype=int32)
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=True
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False

In [ ]:
X_train = tokenizer(
    text=X_train['Impression'].tolist(),
    add_special_tokens=True,
    max_length=158,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_test = tokenizer(
    text=X_test['Impression'].tolist(),
    add_special_tokens=True,
    max_length=158,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_val = tokenizer(
    text=X_val['Impression'].tolist(),
    add_special_tokens=True,
    max_length=158,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [ ]:
for parameter in dict_parameters:
    print(parameter)
    print('Training')
    with mlflow.start_run(): 
        mlflow_log_parameters(parameter) 
        bert_model = create_bert(bert, parameter['learning_rate'], parameter['dropout'])
        hist = bert_model.fit( x = {'input_ids':X_train['input_ids'], 'attention_mask':X_train['attention_mask']}, 
                     y=y_train, epochs=parameter['epochs'], batch_size=parameter['batch_size'])
    
        training_test_mae( mlflow, hist)
        
        print('Testing')
        split_type = 'test'
        y_pred = bert_model.predict(x = {'input_ids':X_test['input_ids'], 'attention_mask':X_test['attention_mask']})
        accuracy, precision, recall, f1score = evaluate(y_test, y_pred, mlflow, split_type)
        
        print('Validation')
        split_type = 'validation'
        y_pred = bert_model.predict(x = {'input_ids':X_val['input_ids'], 'attention_mask':X_val['attention_mask']})
        accuracy, precision, recall, f1score = evaluate(y_val, y_pred, mlflow,  split_type)
        
        mlflow.end_run()